In [ ]:
!pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('HospitalityEmployees.csv', parse_dates=['Date'])
df.head()

In [ ]:
df.info()

In [ ]:
df.set_index('Date', inplace=True)
df.info()

In [ ]:
#a. Постройте график временного ряда для figsize=(12,5)
df['Employees'].plot(figsize = (12,5))

In [ ]:
#b. Добавьте заголовки и подписи в график
xtitle = 'California Hospitality Indystry Employees'
ax=df['Employees'].plot(figsize = (12,5), title = xtitle)
ax.set_xlabel('Date')
ax.set_ylabel('Thousands of Persons')


In [ ]:
#c. Выполните ETS декомпозицию. Подсказка - используйте функцию seasonal_decompose
result=seasonal_decompose (df['Employees'], model='add')
result.plot()

In [ ]:
from statsmodels.iolib.summary import summary
#d. Запустите pmdarima.auto_arima чтобы получить набор параметров
#Это может занять некоторое время, так как необходимо оценить
#множество комбинаций. Подсказка: используйте auto_arima с параметрами seasonal=True, m=12
auto_arima(df['Employees'], seasonal=True, m=12).summary()

In [ ]:
#e. Разделите данные на test и train. Установите последний год (12 записей) для тестирования
len(df)

In [ ]:
train=df.iloc[:336]
test=df.iloc[336:]

In [ ]:
#f. Обучите метод SARIMA на параметрах полученных из пункта d) на
#train['Employees'] и выведите summary получившегося результата.
model = SARIMAX(train['Employees'], order=(1,1,2), seasonal_order=(1,0,1,12))
results=model.fit()
results.summary()

In [ ]:
#g. Получите прогнозируемые значения и нарисуйте графики прогнозируемых и известных значений
xstart=len(train)
xend=len(train)+len(test)-1
predictions=results.predict(start=xstart, end=xend, dynamic=False, typ='levels').rename('Predictions SARIMAX (1, 1, 2)x(1, 0, [1], 12)')

In [ ]:
xtitle='California Hospitality Indystry Employees'
ax=test['Employees'].plot(legend=True, figsize=(12,5), title=xtitle)
predictions.plot(legend=True)
ax.autoscale(axis='x', tight=True)
ax.set_xlabel('Date')
ax.set_ylabel('Thousands of Persons')

In [ ]:
#h. Оцените качество модели с помощью MSE и RMSE.
from sklearn.metrics import mean_squared_error
error=mean_squared_error(test['Employees'], predictions)
print(f'SARIMAX(1,1,2)x(1,0,[1],12) MSE Error: {error:11.10}')

from statsmodels.tools.eval_measures import rmse
error=rmse(test['Employees'], predictions)
print(f'SARIMAX(1,1,2)x(1,0,[1],12) RMSE Error: {error:11.10}')


SARIMAX(1,1,2)x(1,0,[1],12) MSE Error: 77.69121473
SARIMAX(1,1,2)x(1,0,[1],12) RMSE Error: 8.814262007


In [ ]:
#i. Натренируйте модель на полном датасете и предскажите на год вперед.
#Выведите summary получившегося результата
model=SARIMAX(df['Employees'], order=(1,1,2), seasonal_order=(1,0,1,12))
results=model.fit()
fcast=results.predict(len(df), len(df)+12, typ='levels').rename('SARIMAX (1,1,2)x(1,0,[1],12)')
results.summary()

In [ ]:
#j. Постройте график предсказаний
fcast.plot()

In [ ]:
#k. Постройте график предсказаний и график исходных данных.
xtitle='California Hospitality Indystry Employees'
ax=df['Employees'].plot(legend=True, figsize=(12,5), title=xtitle)
fcast.plot(legend=True)
ax.autoscale(axis='x', tight=True)
ax.set_xlabel('Date')
ax.set_ylabel('Thousands of Persons')